In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('drive/My Drive/Colab Notebooks/UMich/EECS595/')
!ls

Mounted at /content/drive
baseline_sentiment_analysis.ipynb    rehydrated_tweets_parquet
emoji_bert_sentiment_analysis.ipynb  res
emotag-dimension-as-emojis.txt	     res_vanil
logs4				     Sentiment_Classification_LSTM_Dec05.ipynb
logs4_vanil			     vanilla_bert_sentiment_analysis.ipynb


In [2]:
#%pip install transformers
%pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 240 kB 7.1 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=528ca55d0719120e65220cac35be37e9b653a7a4d280015643b789bb74e34c9e
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [4]:
import pandas as pd
#from transformers import BertTokenizerFast, BertModel, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
from torch import nn
import emoji as emo
from tqdm import tqdm
import concurrent.futures
import multiprocessing as mp
import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from collections import Counter
import unicodedata as ud
#from transformers import AutoModelForSequenceClassification, AutoTokenizer
from itertools import repeat
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [5]:
SEED = 595

# 1. Load Data

In [ ]:
#parquet_dir = './rehydrated_tweets_parquet'

Get the paths of all parquet files

In [ ]:
#parquet_list = []
#for root, _, files in os.walk(parquet_dir):
#    for file in files:
#        if file[0] != '.':
#            parquet_file_path = os.path.join(root, file)
#            if parquet_file_path[-8:] == '.parquet':
#                parquet_list.append(parquet_file_path)

### Subset Data
We subset data temporarily just for testing and initial temporary results

In [ ]:
# just use the first 2 parquet files
#parquet_list = parquet_list[:2]

Load subset and apply basic pre-processing

In [ ]:
#df = pd.concat(map(pd.read_parquet, tqdm(parquet_list)))
#df = df.reset_index()
#df = df.set_index('index')

100%|██████████| 2/2 [00:03<00:00,  1.76s/it]


In [ ]:
#df

,tweet_date,tweet_id,sentiment,emojis,tweet_day,tweet_month,tweet_year,tweet_year_month,tweet_text
index,,,,,,,,,
0,2019-04-08,1115139461875097600,1,😘,8,4,2019,2019-04,RT @tpagon: ระดับเดซิเบลของเสียงต่างๆ https://...
1,2019-04-08,1115139461875097600,1,😘,8,4,2019,2019-04,GOT7 fighting😘😘😘\n#GOT7 \n#BBMAs\n@GOT7Official
2,2019-04-08,1115139461912731648,1,🤗,8,4,2019,2019-04,I love it when ppl come to me to help with dog...
5,2019-04-08,1115139512231792640,1,😆,8,4,2019,2019-04,Aaaw you're a hella sweet bros😆
6,2019-04-08,1115139516438724608,1,😍,8,4,2019,2019-04,That booty😍😍
...,...,...,...,...,...,...,...,...,...
33970,2019-04-22,1120567851780378624,0,":(,:(",22,4,2019,2019-04,@Psycopathhh_ noted ate :(
33971,2019-04-22,1120567855978811392,1,😝,22,4,2019,2019-04,"Craving something sweet, like donuts😝😝"
33973,2019-04-22,1120567885305245696,1,😘,22,4,2019,2019-04,@paraadonat Miss you too 😘😂


In [6]:
df = pd.read_parquet('./rehydrated_tweets_parquet/2013/01/en_2013_01_29.parquet')
df = df.reset_index()
df = df.set_index('index')
df

,tweet_date,tweet_id,sentiment,emojis,tweet_day,tweet_month,tweet_year,tweet_year_month,tweet_text
index,,,,,,,,,
0,2013-01-29,296150691238518784,1,:),29,1,2013,2013-01,"@ilove_1D143 :) thanks, do you not use them be..."
2,2013-01-29,296150699622952960,0,":(,:(",29,1,2013,2013-01,fresh part that he assumed. The stage lost a f...
3,2013-01-29,296150699622952960,0,":(,:(",29,1,2013,2013-01,@BrettGriffith6 Yay :( I wish I could tho... 
5,2013-01-29,296150703813050368,1,:),29,1,2013,2013-01,@babyqueen700 Kindly follow back :)
6,2013-01-29,296150703829827584,0,😬,29,1,2013,2013-01,@turtle_tiff goodnight 😏😬 and Thankyou . You t...
...,...,...,...,...,...,...,...,...,...
128889,2013-01-29,296513062331109376,0,D;,29,1,2013,2013-01,誰か今日19時からSAやりませんか。
128890,2013-01-29,296513062331109376,0,D;,29,1,2013,2013-01,RT @athkar_ad3ih: اذكار_الصباح ♥\n\n\n\n\n ht...
128891,2013-01-29,296513062331109376,0,D;,29,1,2013,2013-01,i don't want to represent it in front of my cl...


# 2. Split into train and test sets

In [116]:
train_df, test_df = train_test_split(df, test_size = 0.2, random_state = SEED)
test_df_copy = test_df.copy()

In [117]:
pos = [":)",":-)" ,":)" ,":D", "=D",":-]", ":]", ":-3", ":3", ":->", ":>" ,"8-)" ,
       ":-}" ,":}" ,":o)" ,":c)" ,": ˆ)" ,"=]" ,"=)" ,":D" ,"8D" ,"xD" ,"xD" ,"XD" ,
       'XD' ,"=D" ,"=3" ,"BDˆ" ,":-))" ,":’)" ,":’)" ,";)" ,";)" ,"*-)" ,"*)" ,";]", ';]' ,";ˆ)" ,":," ,";D", "<3",
       '😀','😃','😄','😁','😆','😊','😇','😌','😍','🥰','🤩','😘','😗','😚','😙',
       '😋','😝','🤗','🤤','🥳','😎','😺','😸','😻','😽','✌️','👍']
pos_ext = [":)",":-)" ,":)" ,":D", "=D",":-]", ":]", ":-3", ":3", ":->", ":>" ,"8-)" ,
           ":-}" ,":}" ,":o)" ,":c)" ,": ˆ)" ,"=]" ,"=)" ,":D" ,"8D" ,"xD" ,"xD" ,"XD" ,
           'XD' ,"=D" ,"=3" ,"BDˆ" ,":-))" ,":’)" ,":’)" ,";)" ,";)" ,"*-)" ,"*)" ,";]", ';]' ,";ˆ)" ,":," ,";D", "<3",
           '😀','😃','😄','😁','😆','😊','😇','😌','😍','🥰','🤩','😘','😗','😚','😙',
           '😋','😝','🤗','🤤','🥳','😎','😺','😸','😻','😽','✌️','👍','❤️','💕','♥️',
           '💙','💜','💗','🖤','💛','💖','💞','💚','❣️','💓','🧡','💘','💝','💟','🤍','🤎','🫶']
neg =     [':(',':-(',':(', ':’(', ':(', ':c',':<',':<',':[', ':[', ':-||', '>:[', ':{ ',':@', '>:(', 'D’:', 'D:<', 'D:', 'D;', 'D=', ':/', ':/', ':.', '>:/' , ">:/", ":\\", "=/", '=\\', ':L', '=L', ':S', '</3', '<\\',
            '🙃','😐','😑','😶','😒','🙄','😬','😔','🤢','🤮','😵‍💫','😕','🙁','☹️','😟','😧','😨','😰','😢','😭','👎','😖','😣','😞','😓','😩','😫','😠','😡','🤬','😿','🖕']
neg_ext = [':(',':-(',':(', ':’(', ':(', ':c',':<',':<',':[', ':[', ':-||', '>:[', ':{ ',':@', '>:(', 'D’:', 'D:<', 'D:', 'D;', 'D=', ':/', ':/', ':.', '>:/' , ">:/", ":\\", "=/", '=\\', ':L', '=L', ':S', '</3', '<\\',
           '🙃','😐','😑','😶','😒','🙄','😬','😔','🤢','🤮','😵‍💫','😕','🙁','☹️','😟','😧','😨','😰','😢','😭','👎','😖','😣','😞','😓','😩','😫','😠','😡','🤬','😿','🖕','😥','🫤','💔']

In [118]:
pos_names = [emo.demojize(x) for x in pos]
pos_names_ext = [emo.demojize(x) for x in pos_ext]
neg_names = [emo.demojize(x) for x in neg]
neg_names_ext = [emo.demojize(x) for x in neg_ext]

pos_names_replace = [''] * len(pos_names)
pos_names_ext_replace = [''] * len(pos_names_ext)
neg_names_replace = [''] * len(neg_names)
neg_names_ext_replace = [''] * len(neg_ext)

pos_dict = dict(zip(pos_names, pos_names_replace))
pos_dict_ext = dict(zip(pos_names_ext, pos_names_ext_replace))
neg_dict = dict(zip(neg_names, neg_names_replace))
neg_dict_ext = dict(zip(neg_names_ext, neg_names_ext_replace))

def replace_emoticon_with_blank(tweet, emo_dict):
    for key in emo_dict:
        tweet = tweet.replace(key, emo_dict[key])
    return tweet

def naive_sentiment(row):
    if row['no_neg'] and not row['no_pos']:
        return 1
    elif row['no_pos'] and not row['no_neg']:
        return 0
    else:
        return np.random.randint(2)

In [119]:
test_df['tweet_text'] = test_df['tweet_text'].apply(emo.demojize)
test_df['tweet_remove_pos'] = test_df['tweet_text'].apply(replace_emoticon_with_blank, emo_dict=pos_dict)
test_df['no_pos'] = test_df['tweet_text'] == test_df['tweet_remove_pos']
test_df['tweet_remove_neg'] = test_df['tweet_text'].apply(replace_emoticon_with_blank, emo_dict=neg_dict)
test_df['no_neg'] = test_df['tweet_text'] == test_df['tweet_remove_neg']

test_df['naive_senti'] = test_df.apply(naive_sentiment, axis=1)

In [121]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
y_true = test_df['sentiment']
y_pred = test_df['naive_senti']
print(accuracy_score(y_true, y_pred))
print(recall_score(y_true, y_pred))
print(precision_score(y_true, y_pred))
print(f1_score(y_true, y_pred))

0.8251622776927542
0.8030339321357285
0.9224137931034483
0.8585940501088395


In [122]:
test_df = test_df_copy.copy()
test_df['tweet_text'] = test_df['tweet_text'].apply(emo.demojize)
test_df['tweet_remove_pos'] = test_df['tweet_text'].apply(replace_emoticon_with_blank, emo_dict=pos_dict)
test_df['no_pos'] = test_df['tweet_text'] == test_df['tweet_remove_pos']
test_df['tweet_remove_neg'] = test_df['tweet_text'].apply(replace_emoticon_with_blank, emo_dict=neg_dict)
test_df['no_neg'] = test_df['tweet_text'] == test_df['tweet_remove_neg']

test_df['naive_senti'] = test_df.apply(naive_sentiment, axis=1)
y_true = test_df['sentiment']
y_pred = test_df['naive_senti']
print(accuracy_score(y_true, y_pred))
print(recall_score(y_true, y_pred))
print(precision_score(y_true, y_pred))
print(f1_score(y_true, y_pred))

0.8261649691276585
0.8040718562874252
0.9230134726422876
0.8594470046082949
